# 4. Run style transfer at scale
This notebook will take us through what it looks like to run neural style transfer at scale in Azure using Batch AI. 


---

Import utilities. For this notebook, we're going to use some of the utilities provided as part of this package.

In [ ]:
import os
from az.util import bai
%load_ext dotenv
%dotenv

## Setting up your cluster
This section will walk through setting up your cluster and some of the parameters you can use to customize your cluster.

Using the provided utility, set up the Batch AI client:

In [ ]:
client = bai.setup_bai()

Before creating the cluster, we need to set up parameters for our cluster:

In [ ]:
workspace = "<my-workspace>"
cluster_name = "<my-cluster-name>"
cluster_vm_size = "<vm-size>"
cluster_vm_priority = "<vm-priority>"
cluster_minimum_node_count = "<minimum-node-count>"
cluster_maximum_node_count = "<maximum-node-count>"
cluster_initial_node_count = "<initial-node-count>"
cluster_container_mnt_path = "<container-mnt-path>"
admin_user_name = "<username>"
admin_user_password = "<my-secret-password>"

Save the cluster settings to the .env file.

In [ ]:
!dotenv set WORKSPACE $workspace
!dotenv set CLUSTER_NAME $cluster_name
!dotenv set CLUSTER_VM_SIZE $cluster_vm_size
!dotenv set CLUSTER_VM_PRIORITY $cluster_vm_priority
!dotenv set CLUSTER_MINIMUM_NODE_COUNT $cluster_minimum_node_count
!dotenv set CLUSTER_MAXIMUM_NODE_COUNT $cluster_maximum_node_count
!dotenv set CLUSTER_INITIAL_NODE_COUNT $cluster_initial_node_count
!dotenv set CLUSTER_CONTAINER_MNT_PATH $cluster_container_mnt_path
!dotenv set ADMIN_USER_NAME $admin_user_name
!dotenv set ADMIN_USER_PASSWORD $admin_user_password

In [ ]:
%reload_ext dotenv
%dotenv

Using the Batch AI client, we can set up the cluster. In Batch AI, clusters must belong within a _workspace_. Next, create both the workspace and cluster: 

In [ ]:
bai.create_workspace(client, ws=os.getenv('WORKSPACE'))
bai.create_autoscale_cluster(client, os.getenv('CLUSTER_NAME'))

Take a look at the state of the cluster.

NOTE: This can also be done by going into the portal and inspecting the cluster via the Batch AI UI.

In [ ]:
cluster = bai.get_cluster(client, name=os.getenv('CLUSTER_NAME'), ws=os.getenv('WORKSPACE'))
print(('Cluster state: {0}; Allocated: {1}; Idle: {2}; ' +
     'Unusable: {3}; Running: {4}; Preparing: {5}; ' +
     'Leaving: {6}').format(
        cluster.allocation_state,
        cluster.current_node_count,
        cluster.node_state_counts.idle_node_count,
        cluster.node_state_counts.unusable_node_count,
        cluster.node_state_counts.running_node_count,
        cluster.node_state_counts.preparing_node_count,
        cluster.node_state_counts.leaving_node_count))

## Running a job on the cluster
This section of the notebook will walk through how to run a Batch AI job on the cluster we just created. This section will first use AzCopy to upload our individual frames from the video onto blob storage. It will also use AzCopy to copy over the style transfer script and the style image.

After that, we will primarily be relying on the script `create_job.py` to create jobs on Batch AI.

First we need to define some variables for Azure storage (which we'll treat as our fileshare):

- `fs_script` - what to name the script that is uploaded to storage
- `fs_style_image` - what to name the style image that is uploaded to storage
- `fs_content_dir` - what to name the directory that we'll store all the content images we want to apply style transfer to

In [ ]:
fs_script = "script.py"
fs_style_image = "style_image.py"
fs_content_dir = "orangutan"

Persist these variables to our `.env` file

In [ ]:
!dotenv set FS_SCRIPT $fs_script
!dotenv set FS_STYLE_IMAGE $fs_style_image
!dotenv set FS_CONTENT_DIR $fs_content_dir

Reload our environment variables from our `.env` file so we can use these variables as enviroment variables.

In [ ]:
%reload_ext dotenv
%dotenv

First we use AzCopy to upload the following items into the storage account we've created:

- `style_transfer_script.py` - this is the style transfer script that we've already tested locally. We'll upload this script as `fs_script`
- `sample_renior.jpg` - this is the style image that the style transfer script will use. We'll upload this script as `fs_style_image`
- `/orangutan` - this is the directory with all the individual frames. We'll name the directory once uploaded to storage as `fs_content_dir`

In [ ]:
%%bash
azcopy \
    --source pytorch/style_transfer_script.py \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_SCRIPT} \
    --dest-key $STORAGE_ACCOUNT_KEY
    
azcopy \
    --source pytorch/images/style_images/sample_renior.jpg \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_STYLE_IMAGE} \
    --dest-key $STORAGE_ACCOUNT_KEY
    
azcopy \
    --source pytorch/images/${VIDEO_NAME} \
    --destination https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME}/${FS_CONTENT_DIR} \
    --dest-key $STORAGE_ACCOUNT_KEY \
    --recursive

Now that we have the images and scripts that Batch AI will use uploaded to storage, we need to setup our Batch AI job. First we define some variables for the job:

- `experiment_prefix` - experiments are a logical container for Batch AI jobs. This is the prefix for your experiment name.
- `job_name_prefix` - this is the individual Batch AI job. this is the prefix for the jobs name.
- `job_node_count` - this number defines the number of jobs to run on a single node.
- `job_batch_size` - this number defines how many images to process on a single job.
- `fs_output_dir_prefix` - all output images will be saved into a folder in blob. This variable is the prefix for the name of that folder.
- `fs_logger_dir_prefix` - all jobs will output log files into a folder in blob. This variable is the prefix for the name of that folder.

In [ ]:
experiment_prefix = "exp"
job_name_prefix = "job"
job_node_count = 1
job_batch_size = 50
fs_output_dir_prefix = "output"
fs_logger_dir_prefix = "log"

Persist these variables to our `.env` file

In [ ]:
!dotenv set EXPERIMENT_PREFIX $experiment_prefix
!dotenv set JOB_NAME_PREFIX $job_name_prefix
!dotenv set JOB_NODE_COUNT $job_node_count
!dotenv set JOB_BATCH_SIZE $job_batch_size
!dotenv set FS_OUTPUT_DIR_PREFIX $fs_output_dir_prefix
!dotenv set FS_LOGGER_DIR_PREFIX $fs_logger_dir_prefix

Reload our environment variables from our `.env` file so we can use these variables as enviroment variables.

In [ ]:
%reload_ext dotenv
%dotenv

Using the `create_job.py` file, lets kick off our Batch AI jobs.

In [ ]:
!python az/create_job.py

When the jobs finish running, you can use the Azure portal or Storage explorer to inspect the output images.

Inside your Blob Container, you should notice that a new directory with the datetime-stamp is created. Output images are stored there.

---

## Conclusion
In this notebook, we've configured a bunch of settings for what the cluster should look like, how the files in storage should be structured, and what the job parameters should be. You can feel free to configure and play around with these parameters to make sure that they work best for your workload. 

With these parameters set, we've also created an autoscaling cluster in Batch AI, uploaded scripts and images into Azure blob storage, and used those scripts and images to perform a batch style transfer job at scale with the `create_job.py` script.

Next, we're going to [setup our Docker image and run the Batch AI jobs from there.](./05_run_the_batch_ai_job_from_docker.ipynb)